
# Fine-tuning Llama:3.2:1B for Information Extraction

Most of the business tasks require information extraction and almost every task can be handled with small models if you fine-tune the model for your specific needs. In this cookbook, we'll fine-tune Llama:3.2:1B for information extraction.

### Installation

We are going to use Unsloth for fine-tuning our model.

In [59]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

And we will use ollama the fine-tuned model.

In [60]:
!curl https://www.ollama.com/install.sh | OLLAMA_VERSION="0.5.4" sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13269    0 13269    0     0  31237      0 --:--:-- --:--:-- --:--:-- 31221
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [61]:
import subprocess
import os
subprocess.Popen(['ollama', 'serve'], preexec_fn=os.setpgrp)
import time
time.sleep(3)

In [62]:
!ollama --version

ollama version is 0.5.4


In [63]:
# Our base model is llama3.2:1b, so let's pull that model using ollama
%%capture
!ollama pull llama3.2:1b

### Finetuning Llama3.2-1B using unsloth

In [64]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# See other models you can fine-tune by just changing the model name.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",
    "unsloth/Mistral-Small-Instruct-2409",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",
    "unsloth/Llama-3.2-1B-bnb-4bit",
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit"
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.1.6: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

## Formatting Dataset

Now we can upload our fine-tuning dataset we generated with Dria and put it into the right format for fine-tuning.


In [67]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    # Convert the dataset format to match the expected conversation structure
    conversations = []
    for example in examples:
        # Each example is already a list of messages
        conversations.append(example)

    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in conversations]
    return {"text": texts}

# Load the dataset from the JSON file
from datasets import load_dataset
dataset = load_dataset("json", data_files="conversation_format_dataset.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [68]:
from unsloth.chat_templates import get_chat_template, standardize_sharegpt
from datasets import load_dataset
import json

# 1. Initialize tokenizer with chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

# 2. Define formatting function
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return {"text": texts}

# 3. Load and preprocess dataset
# First read your JSON file
with open('conversation_format_dataset.json', 'r') as f:
    raw_data = json.load(f)

# Format the data to include 'conversations' key
formatted_data = [{'conversations': conv} for conv in raw_data]

# Save formatted data
with open('formatted_dataset.json', 'w') as f:
    json.dump(formatted_data, f)

# Now load the formatted dataset
dataset = load_dataset("json", data_files="formatted_dataset.json", split="train")

# Standardize the format
dataset = standardize_sharegpt(dataset)

# Apply formatting function
dataset = dataset.map(formatting_prompts_func, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Standardizing format:   0%|          | 0/211 [00:00<?, ? examples/s]

Map:   0%|          | 0/211 [00:00<?, ? examples/s]

## Intializing Training

In [69]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [129]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 7,
        num_train_epochs = 3, # Set this for 1 full training run.
        #max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 3,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [130]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [131]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 211 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 78
 "-____-"     Number of trainable parameters = 11,272,192


Step,Training Loss
3,0.446100
6,0.525800
9,0.611700
12,0.919600
15,0.886100
18,0.874400
21,0.839700
24,1.006800
27,1.048600
30,0.764100


config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [134]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [135]:
# Save to 8bit Q8_0
if True: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.53 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 16/16 [00:00<00:00, 40.59it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be /content/model/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {32}
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model.bin'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> Q8_0, shape = 

In [136]:
!ollama create information_extractor -f ./model/Modelfile

transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠸ transferring model data ⠼ transferring model data ⠦ transferring model data ⠧ transferring model data ⠧ transferring model data ⠏ transferring model data ⠋ transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠼ transferring model data ⠦ transferring model data ⠧ transferring model data ⠇ transferring model data ⠇ transferring model data ⠋ transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠦ transferring model data ⠦ transferring model data ⠇ transferring model data ⠏ transferring model data ⠏ transferring model data ⠋ transferring model data ⠹ transferring model data ⠹ transferring model data ⠼ transferring model data ⠼ transferring model data ⠴ transferring model data ⠧ transferring model data ⠧ transferring

In [137]:
!ollama ls

NAME                              ID              SIZE      MODIFIED               
information_extractor:latest      b19d698b1f4a    1.3 GB    Less than a second ago    
information_extractor_4:latest    b073552fba5f    1.3 GB    21 minutes ago            
information_extractor_3:latest    bcfa5621d6ec    1.3 GB    52 minutes ago            
llama3.2:1b                       baf6a787fdff    1.3 GB    2 hours ago               
information_extractor_2:latest    046dd9e64903    1.3 GB    2 hours ago               


### We'll test our model with different inputs.

We added two examples
- Product highlights extraction
- News details extraction

However, you can try with different examples.

### Try with Stargate Project Announcement

In [138]:
news ="""
The Stargate Project is a new company which intends to invest $500 billion over the next four years building new AI infrastructure for OpenAI in the United States. We will begin deploying $100 billion immediately. This infrastructure will secure American leadership in AI, create hundreds of thousands of American jobs, and generate massive economic benefit for the entire world. This project will not only support the re-industrialization of the United States but also provide a strategic capability to protect the national security of America and its allies.

The initial equity funders in Stargate are SoftBank, OpenAI, Oracle, and MGX. SoftBank and OpenAI are the lead partners for Stargate, with SoftBank having financial responsibility and OpenAI having operational responsibility. Masayoshi Son will be the chairman.

Arm, Microsoft, NVIDIA, Oracle, and OpenAI are the key initial technology partners. The buildout is currently underway, starting in Texas, and we are evaluating potential sites across the country for more campuses as we finalize definitive agreements.

As part of Stargate, Oracle, NVIDIA, and OpenAI will closely collaborate to build and operate this computing system. This builds on a deep collaboration between OpenAI and NVIDIA going back to 2016 and a newer partnership between OpenAI and Oracle.

This also builds on the existing OpenAI partnership with Microsoft. OpenAI will continue to increase its consumption of Azure as OpenAI continues its work with Microsoft with this additional compute to train leading models and deliver great products and services.

All of us look forward to continuing to build and develop AI—and in particular AGI—for the benefit of all of humanity. We believe that this new step is critical on the path, and will enable creative people to figure out how to use AI to elevate humanity.


"""

In [140]:
import requests
import json

url = "http://localhost:11434/api/chat"

payload = {
    "model": "information_extractor",
    "messages": [
        {
            "role": "system",
            "content": "You are an information extractor. "
        },
        {
            "role": "user",
            "content": f"Get me the summary and the key financial information from the given context. Here is the context to extract information from: {news}"
        }
    ],
    "stream": False
}

response = requests.post(url, json=payload)
response.raise_for_status()

# The entire assistant response is in a single JSON
data = response.json()
assistant_content = data["message"]["content"]

print("Assistant:\n", assistant_content)


Assistant:
 <{"summary":"The Stargate Project aims to secure American leadership in AI, create thousands of jobs, and generate substantial economic benefits. It involves significant investments from leading companies like SoftBank, OpenAI, Oracle, and MGX, with a focus on AI infrastructure and technology partnerships."} {"financial_info": {"investment": "$500 billion", "deployment_strategy": ["$100 billion immediate deployment", "$100 billion over four years"]}, "key_partnerships": [{"name": "SoftBank", "role": "Financial Responsibility"}, {"name": "OpenAI", "role": "Operational Responsibility"}], "impact": {"economic": "Millions of dollars in economic benefits and job creation", "security": "Strategic capability to protect national security"}, "strategy_execution_status": "Buildout underway, evaluating site options for further expansion."}


### Try with Product Description Example

The product information taken by JBL's official website.

In [141]:
product = """

**JBL Tour One M2**

$299.95 each


What the experts say
--------------------

As Hypebeast says, the JBL Tour ONE M2 builds upon the company’s hybrid True Adaptive ANC technology with new 40mm dynamic drivers for premium sound.
-----------------------------------------------------------------------------------------------------------------------------------------------------

"With a fantastic listening experience, comfortable fit and competitive battery life, the JBL Tour One M2 is a great pair of over-ear headphones" Stuff says.
-------------------------------------------------------------------------------------------------------------------------------------------------------------

As Trusted Reviews says "JBL's flagship wireless over-ears offer an enormous number of features to make the commute more enjoyable, with fine sound, effective noise-cancellation and an excellent wireless performance."
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

"The perfect headphones for longer trips and/or regular tours on the bus, train or subway" Lite mentions.
---------------------------------------------------------------------------------------------------------

"JBL work its sound-enhancing magic to create a truly feature-packed pair of over-ear noise cancelling headphones" Expertreviews mentions
-----------------------------------------------------------------------------------------------------------------------------------------

5 STAR REVIEW CINEMA CHOICE
---------------------------

True Adaptive Noise Cancelling
------------------------------

Enjoy the soundtrack of your life no matter where you are.

True Adaptive Noise Cancelling technology automatically uses 4 noise sensing mics to adjust to your surroundings in real-time, minimizing distractions when you need to focus and optimizing the Tour’s performance for a superior audio experience—all while you adjust ANC mode control with the JBL Headphones app.

**Adaptive Noise Cancelling**

Noise Cancelling off

Noise Cancelling on

**Scenarios**

Café

On the road

Airplane

Metro





For the best experience, use headphones.



Smart Ambient Sound
-------------------

Stay alert to the sounds of the world around you while enjoying your music with Smart Ambient, controlling the ambient sound levels and turning the Personal Sound Amplification button on through the JBL Headphones App to boost the conversations' volume with +15-20dB and adjust the L/R balance.

Turn on _Personal Sound Amplification_ and amplify ambient sounds with +15-20dB, adjust the L/R balance.



4-Mic Superior Calls
with Voice Control
-----------------------------------------

The 4-mic crystal call algorithm of the JBL Tour One M2 allows you to have better conversations even in loud or windy environments or to get help from your preferred voice assistant. And with VoiceAware, you can choose how much of your own voice to hear by controlling the amount of mic input routed back into your earbuds.

Smart Talk for Seamless Chats
-----------------------------

Thanks to the Smart Talk feature, you don't need to pause the music to have a quick conversation: its advanced voice recognition technology reacts to your voice pausing the music and enabling TalkThru.

Stereo Spatial Audio

Immersive JBL
Spatial Sound
-----------------------------

Immerse yourself in JBL Spatial Sound and feel like being in a theatre, at a concert or in the studio with your favorite artist. Thanks to this proprietary technology, sound comes from virtually anywhere around you, no matter the source or the device you are playing from



Low Volume Dynamic EQ
---------------------

Protect your hearing without compromising your listening experience. Even at low volume, JBL Tour One M2's optimized EQ boosts both low and high frequencies to maintain high-fidelity performance.



Up to 50 Hours of Playback
--------------------------

Listen without limits. Enjoy up to 50 hours of playback, or up to 30 hours with True Adaptive Noise Cancelling enabled. And when you need a boost, just speed charge for 10 minutes and get an extra 5 hours of battery life.

Personi-fi 2.0
--------------

Hear and feel every note of your playlist. Shape your audio to match your personal listening profile with interactive customization tools that use precise information about you and your preferences to provide the exact sound curve that suits you best.

[Learn more about Personi-fi](https://www.youtube.com/watch?v=vSPgN1apktQ&ab_channel=JBL)






### True Adaptive Noise Cancelling with Smart Ambient

Enjoy the soundtrack of your life no matter where you are. True Adaptive Noise Cancelling technology automatically uses 4 noise sensing mics to adjust to your surroundings in real-time, minimizing distractions when you need to focus, optimizing the JBL Tour One M2's performance for a superior audio experience—all while you adjust ANC mode control with the JBL Headphones app.



### 4-mic superior calls with voice control

The 4-mic crystal call algorithm of the JBL Tour One M2 allows you to have better conversations even in loud or windy environments or to get help from your preferred voice assistant. And with VoiceAware, you can choose how much of your own voice to hear by controlling the amount of mic input routed back into your headphones.



### Smart Ambient technology

Stay alert to the sounds of the world around you while enjoying your music with Smart Ambient, controlling the ambient sound levels and turning the Personal Sound Amplification button on through the JBL Headphones App to boost the conversations' volume with +15-20dB and adjust the L/R balance.

Features
--------

### Smart Talk for seamless chats

Thanks to the Smart Talk feature, you don't need to pause the music to have a quick conversation: its advanced voice recognition technology reacts to your voice pausing the music and enabling TalkThru.

### Legendary Pro Sound

Thanks to extensive global research and testing, the superior JBL sound curve delivers precise, accurate audio and less distortion with all the music you love at any frequency or volume. The Tour One M2 40mm drivers let you experience the ultimate sound performance, from rich, punchy bass to thrilling top notes.

### Immersive JBL Spatial Sound

Immerse yourself in JBL Spatial Sound and feel like being in a theatre, at a concert or in the studio with your favorite artist. Thanks to this proprietary technology, sound comes from virtually anywhere around you, no matter the source or the device you are playing from.

### Personi-Fi 2.0

Hear and feel every note of your playlist. Shape your audio to match your personal listening profile with interactive customization tools that use precise information about you and your preferences to provide the exact sound curve that suits you best.

### Hands free voice control

Stay connected to your digital world anywhere. Just say "Hey Google" or "Alexa" to speak to your preferred Voice Assistant and control your headphones with voice prompts in multiple languages.

### Fast Pair enabled by Google and Microsoft swift pair

Discover Bluetooth accessories in close proximity to your device—then connect with a single tap. With Microsoft swift pair you can seamlessly pair your Bluetooth devices with your Windows 10 or higher PC.

### Stay in control with the JBL Headphones app

Personalize the soundtrack of your life. Just download the free JBL Headphones app to customize EQ, switch on Adaptive Noise Cancelling, adjust your ANC level or ambient sound control settings, and check your best fit. You can also customize how the headphones behave through the Touch customization or adapt the sound output to audio or video to get the most out of your content, choose your preferred voice assistant and much more.

### Bluetooth 5.3 LE audio\*

The JBL Tour One M2 feature Bluetooth 5.3, for a more stable and secure connection, so that you can enjoy wireless connection at its best. Featuring the last Bluetooth technology, your JBL Tour One M2 become virtually futureproof. \*Available via OTA update at a later stage.



"""

In [142]:
import requests
import json

url = "http://localhost:11434/api/chat"

payload = {
    "model": "information_extractor",
    "messages": [
        {
            "role": "system",
            "content": "You are an information extractor. "
        },
        {
           "role": "user",
            "content": f" Extract the features of the given product. Here is the context to extract information from: {product}"
        }
    ],
    "stream": False
}

response = requests.post(url, json=payload)
response.raise_for_status()

# The entire assistant response is in a single JSON
data = response.json()
assistant_content = data["message"]["content"]

print("Assistant:\n", assistant_content)


Assistant:
 <{"Features": [{"Key Features": ["True Adaptive ANC","Smart Ambient","Smart Talk","Immersive Spatial Sound","Personi-fi 2.0"],"Audio Qualities": {"Sound Enhancement": "Legendary Pro Sound","Dynamic Range": "Efficient use of low and high frequencies","Sound Pressure Wave Efficiency": "High efficiency"}, "Battery Life": {"Standard Battery Life": "50 Hours","Enhanced with Adaptive Noise Cancelling": "Up to 60 Hours"}, "Connectivity": ["Bluetooth 5.3","Google Swift Pair","Microsoft Swift Pair"]}, "Pricing": {"Price": "$299.95","Discounts": null}] }
